In [1]:
from GenZ import decode_moddeling, prefill_moddeling

import pandas as pd
from plotnine import *
import plotnine as p9
from GenZ.system import System
import pandas as pd
from GenZ.analyse_model import *


In [2]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2   

In [3]:
model_df, summary_table = decode_moddeling(model = 'grok-1', batch_size = 1, Bb = 1 , bits='int8',
                            input_tokens = 1, output_tokens = 0, model_profilling=True,
                            tensor_parallel = 1, pipeline_parallel = 1, debug=True)
display(model_df)
display(summary_table)
print(f'Model Weights={summary_table["Total Weights (MB)"][0]/1024/1024} T')
print(f'Attn Size:{(model_df.loc[0,"Input_w (MB)"] + model_df.loc[5,"Input_w (MB)"])*120/1024} B')
print(f'Each MLP Size:{sum(model_df.loc[6:,"Input_w (MB)"])*120/1024/16} B')
# print(f'FWD pass MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120/1024/16} B')

/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_decode.py:62: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 293.25 GB, KV Cache:0.0001220703125 


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,Repeat,64,0,0.0,0.0,0.0,0.0,0.0
1,GEMM,"[((1, 6144, 1), (8192, 6144), (1, 8192, 1))]",0.999715,50.331648,0.005859,48.0,0.007812,48.013672
2,Logit,"((1, 48, 1, 128), (1, 8, 1, 128), (1, 48, 1, 1))",0.851441,0.006144,0.005859,0.000977,0.000046,0.006882
3,Logit,"((1, 48, 1, 128), (1, 8, 0, 128), (1, 48, 1, 0))",0.0,0.0,0.005859,0.0,0.0,0.005859
4,Attend,"((1, 48, 1, 1), (1, 8, 1, 128), (1, 48, 1, 128))",0.851441,0.006144,0.000046,0.000977,0.005859,0.006882
5,Attend,"((1, 48, 1, 0), (1, 8, 0, 128), (1, 48, 1, 128))",0.0,0.0,0.0,0.0,0.005859,0.005859
6,GEMM,"[((1, 6144, 1), (6144, 6144), (1, 6144, 1))]",0.999675,37.748736,0.005859,36.0,0.005859,36.011719
7,GEMM,"[((1, 6144, 1), (98304, 6144), (1, 98304, 1))]",0.999827,603.979776,0.005859,576.0,0.09375,576.099609
8,GEMM,"[((1, 6144, 0), (294912, 6144), (1, 294912, 0))]",0.0,0.0,0.0,1728.0,0.0,1728.0
9,GEMM,"[((1, 98304, 1), (6144, 98304), (1, 6144, 1))]",0.999827,603.979776,0.09375,576.0,0.005859,576.099609


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB)
0,82947.342336,300304.005859,300288.0,3456.0,0.125,1728.0


Model Weights=0.286376953125 T
Attn Size:0.0 B
Each MLP Size:34.013671875 B


In [4]:
for model in ['grok-1']:
    model_df, summary_table = prefill_moddeling(model = model, batch_size = 1,
                            input_tokens = 1024, model_profilling=True,
                            tensor_parallel = 1, pipeline_parallel = 1)
    print(model)
    display(model_df)
    display(summary_table)


grok-1


/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_prefill.py:58: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 586.5 GB, KV Cache:0.25 


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,Repeat,64,0,0.0,0.0,0.0,0.0,0.0
1,GEMM,"[((1, 6144, 1024), (8192, 6144), (1, 8192, 102...",396.387097,51539.607552,12.0,96.0,16.0,124.0
2,Logit,"((1, 48, 1024, 128), (1, 8, 1024, 128), (1, 48...",55.854545,6442.450944,12.0,2.0,96.0,110.0
3,Attend,"((1, 48, 1024, 1024), (1, 8, 1024, 128), (1, 4...",55.854545,6442.450944,96.0,2.0,12.0,110.0
4,GEMM,"[((1, 6144, 1024), (6144, 6144), (1, 6144, 102...",384.0,38654.705664,12.0,72.0,12.0,96.0
5,GEMM,"[((1, 6144, 256), (393216, 6144), (1, 393216, ...",122.803248,618475.290624,3.0,4608.0,192.0,4803.0
6,GEMM,"[((1, 393216, 256), (6144, 393216), (1, 6144, ...",122.803248,618475.290624,192.0,4608.0,3.0,4803.0
7,EndRepeat,64,0,0.0,0.0,0.0,0.0,0.0


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB)
0,8.576191e+07,642944.0,600576.0,0,256.0,4803.0


In [5]:
for model in ['llama_405b']:
    model_df, summary_table = decode_moddeling(model = 'llama_405b', batch_size = 1, Bb = 1 , bits='int8',
                                input_tokens = 1, output_tokens = 0, model_profilling=True,
                                tensor_parallel = 1, pipeline_parallel = 1, debug=True) 
    display(model_df)
    display(summary_table)
    print(f'Model Weights={summary_table["Total Weights (MB)"][0]/1024/1024} T')
    print(f'Attn Size:{(model_df.loc[0,"Input_w (MB)"] + model_df.loc[5,"Input_w (MB)"])*120/1024} B')
    print(f'Each MLP Size:{sum(model_df.loc[6:,"Input_w (MB)"])*120/1024/16} B')


/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_decode.py:62: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 378.0 GB, KV Cache:0.00048065185546875 


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,Repeat,126,0,0.0,0.0,0.0,0.0,0.0
1,GEMM,"[((1, 16384, 1), (20480, 16384), (1, 20480, 1))]",0.99989,335.54432,0.015625,320.0,0.019531,320.035156
2,Logit,"((1, 128, 1, 128), (1, 16, 1, 128), (1, 128, 1...",0.882759,0.016384,0.015625,0.001953,0.000122,0.0177
3,Logit,"((1, 128, 1, 128), (1, 16, 0, 128), (1, 128, 1...",0.0,0.0,0.015625,0.0,0.0,0.015625
4,Attend,"((1, 128, 1, 1), (1, 16, 1, 128), (1, 128, 1, ...",0.882759,0.016384,0.000122,0.001953,0.015625,0.0177
5,Attend,"((1, 128, 1, 0), (1, 16, 0, 128), (1, 128, 1, ...",0.0,0.0,0.0,0.0,0.015625,0.015625
6,GEMM,"[((1, 16384, 1), (16384, 16384), (1, 16384, 1))]",0.999878,268.435456,0.015625,256.0,0.015625,256.03125
7,GEMM,"[((1, 16384, 1), (106496, 16384), (1, 106496, ...",0.99993,1744.830464,0.015625,1664.0,0.101562,1664.117188
8,GEMM,"[((1, 53248, 1), (16384, 53248), (1, 16384, 1))]",0.99992,872.415232,0.050781,832.0,0.015625,832.066406
9,EndRepeat,126,0,0.0,0.0,0.0,0.0,0.0


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB)
0,405878.53824,387111.897949,387072.0,0,0.492188,1664.117188


Model Weights=0.369140625 T
Attn Size:0.0 B
Each MLP Size:20.15625 B
